In [7]:
from dotenv import load_dotenv
import os

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core.node_parser import SemanticSplitterNodeParser, SentenceSplitter
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, Document
from llama_index.vector_stores.milvus import MilvusVectorStore

from llama_index.embeddings.nvidia import NVIDIAEmbedding
from llama_index.llms.nvidia import NVIDIA


c:\Users\kthk1\Documents\NUS\AI_Singapore\Nvidia\Nvidia-RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# from openai import OpenAI
# load_dotenv()

# client = OpenAI(
#   api_key= os.getenv("NIM_APIKEY"),
#   base_url="https://integrate.api.nvidia.com/v1"
# )

# response = client.embeddings.create(
#     input=["What is the capital of France?"],
#     model="nvidia/nv-embedqa-mistral-7b-v2",
#     encoding_format="float",
#     extra_body={"input_type": "query", "truncate": "NONE"}
# )

# print(response.data[0].embedding)


[0.003818511962890625, 0.0019378662109375, -0.00588226318359375, -0.007709503173828125, 0.022247314453125, 0.00989532470703125, -0.01678466796875, -0.035247802734375, 0.0207061767578125, 0.0235748291015625, 0.004669189453125, -0.006534576416015625, -0.020599365234375, 0.04034423828125, -0.0152740478515625, 0.0245361328125, -0.0262603759765625, 0.01873779296875, -0.00909423828125, -0.01462554931640625, 0.00749969482421875, 0.0082550048828125, -0.0211029052734375, -0.0029926300048828125, -0.00937652587890625, 0.004192352294921875, 0.0208587646484375, -0.004535675048828125, 0.0182342529296875, 0.0124053955078125, -0.01061248779296875, -0.0121917724609375, -0.015869140625, 0.00823211669921875, -0.027313232421875, 0.007259368896484375, 0.006702423095703125, -0.007572174072265625, 0.008544921875, -0.0194854736328125, -0.01593017578125, -0.0202484130859375, 0.00884246826171875, -0.01131439208984375, -0.0204925537109375, -0.0248870849609375, 0.016754150390625, -0.00563812255859375, 0.003044128

In [88]:
import getpass
import os

# del os.environ['NVIDIA_API_KEY']  ## delete key and reset
if os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    print("Valid NVIDIA_API_KEY already in environment. Delete to reset")
else:
    nvapi_key = getpass.getpass("NVAPI Key (starts with nvapi-): ")
    assert nvapi_key.startswith(
        "nvapi-"
    ), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key
    
embedder = NVIDIAEmbedding(model="NV-Embed-QA", truncate="END")

nim_llm = NVIDIA(model="mistralai/mistral-7b-instruct-v0.2")

Valid NVIDIA_API_KEY already in environment. Delete to reset


In [14]:
vector_store = MilvusVectorStore(
            dim=1024,
            collection_name="milvus_test",
            uri="http://localhost:19530/",
            overwrite=True
        )

chunks = chunk_document(documents)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_store_index = VectorStoreIndex(nodes=chunks,vector_store=vector_store, embed_model=embed_model, storage_context=storage_context)

NameError: name 'embedder' is not defined

In [9]:
embed_model = AzureOpenAIEmbedding(
    engine="text-embedding-ada-002",
    model="text-embedding-ada-002",
    api_version="2024-07-01-preview",
    azure_endpoint=os.getenv('AZURE_3.5_ENDPOINT'),
    api_key=os.getenv('AZURE_3.5_API_KEY')
)

llm = AzureOpenAI(
    engine="gpt-35-turbo",
    api_version="2024-07-01-preview",
    azure_endpoint=os.getenv('AZURE_3.5_ENDPOINT'),
    api_key=os.getenv('AZURE_3.5_API_KEY'),
    model="gpt-35-turbo",
    temperature=0,
    top_p=0
)

In [10]:
reader = SimpleDirectoryReader(input_files=["./../data/test.pdf"])
documents = reader.load_data()

In [11]:
documents

[Document(id_='9969a1ba-97bd-4c7e-9ea3-ca91a8e9d077', embedding=None, metadata={'page_label': '385', 'file_name': 'test.pdf', 'file_path': '..\\data\\test.pdf', 'file_type': 'application/pdf', 'file_size': 527692, 'creation_date': '2024-10-12', 'last_modified_date': '2024-10-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="Annals of Oncology 10: 385-390, 1999.\n© 1999 Kluwer Academic Publishers. Printed  in the Netherlands.\nSpecial article\nEssential drugs for cancer therapy:  A World Health Organization\nconsultation\nK. Sikora,1 S. Advani,2 V. Koroltchouk,1 I. Magrath,3 L. Levy,4 H. Pinedo,5 G. Schwartsmann,6\nM.Tattersall7 & S.Yan8\n1 WHO Cancer Programme, International Agency for Research on Cancer, Lyon, France; 2Department of Medical Oncology

In [15]:
def chunk_document(documents):

    base_splitter = SentenceSplitter(chunk_size=512)
    splitter = SemanticSplitterNodeParser(
        buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model, base_splitter=base_splitter
    )
    chunks = splitter.get_nodes_from_documents(documents)

    return chunks

# Chunk your documents before passing them to the index + Flatten the list of chunks
# chunked_data = [chunk for documents in documents for chunk in chunk_document(documents)]

In [16]:
chunks = chunk_document(documents)

In [18]:
chunks[0].text

'Annals of Oncology 10: 385-390, 1999.\n© 1999 Kluwer Academic Publishers. '

In [ ]:
Document()

In [22]:
[Document(text=chunk.text) for chunk in chunks]

[Document(id_='1f92cdfe-7418-4416-8c0a-a6bc0b5f4ec7', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Annals of Oncology 10: 385-390, 1999.\n© 1999 Kluwer Academic Publishers. ', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='b9dc169d-fc64-4ff2-8d7d-70875a6df27e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Printed  in the Netherlands.\nSpecial article\nEssential drugs for cancer therapy:  A World Health Organization\nconsultation\nK. Sikora,1 S. Advani,2 V. Koroltchouk,1 I. Magrath,3 L. Levy,4 H. Pinedo,5 G. Schwartsmann,6\nM.Tattersall7 & S.Yan8\n1 WHO Cancer Programme, International Agency for Research on Cancer, Lyon, France; 2Department of Medical Oncology, Tata Memorial\nHospital, India; 3National Cancer 

In [85]:
vector_store = MilvusVectorStore(
            dim=1024,
            collection_name="milvus_test",
            uri="http://localhost:19530/",
            overwrite=True
        )

chunks = chunk_document(documents)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_store_index = VectorStoreIndex(nodes=chunks,vector_store=vector_store, embed_model=embedder, storage_context=storage_context)

In [90]:
query_engine = vector_store_index.as_query_engine(llm=nim_llm)
res = query_engine.query("What are the different categories of tumour?")
print(res)

 Based on the provided context, tumours can be categorized into five groups:

1. Tumours for which the use of a single or a combination of drugs used alone or with other therapeutic modalities will result in a cure as defined by a normal life-span in some and a prolongation of survival in most patients.
2. Tumours where the average survival is prolonged when chemotherapy is used as an adjuvant to local surgery or radiotherapy in early stages of disease.
3. Tumours for which there is evidence that the use of one drug or a combination of drugs will cause tumour shrinkage in more than 20% of selected patients with metastatic disease, with almost certain improvement in quality of life. Prolongation of survival occurs in most of the responding patients but may be of short duration.
4. Tumours where local control may be improved by the use of chemotherapy before, during or after surgery and radiotherapy.
5. Tumours for which there are currently no effective drugs. Objective responses occur i